<a href="https://colab.research.google.com/github/rashida048/BigDataAnalytics/blob/master/partitioning_and_treeAggregating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 60 kB/s 
     |████████████████████████████████| 198 kB 55.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=60aa1db975f953b419d6f1e9f0ac17f3dc764c6b0fcd7cb868298533ece4e826
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 

In [4]:
pairs = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8]).map(lambda x: (x, x))
pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)]

In [5]:
pairs.partitionBy(4).glom().collect()

[[(4, 4), (8, 8)], [(1, 1), (5, 5)], [(2, 2), (6, 6)], [(3, 3), (7, 7)]]

In [6]:
counts = pairs.count()
counts

8

In [7]:
numberOfBatches = 4
partitionedRDD = pairs.partitionBy(int(counts/numberOfBatches)).glom()

partitionedRDD.cache()

for i in range(4):
  sets = partitionedRDD.map(lambda x: x[i]).collect()
  print("Batch number ", i, sets)

Batch number  0 [(2, 2), (1, 1)]
Batch number  1 [(4, 4), (3, 3)]
Batch number  2 [(6, 6), (5, 5)]
Batch number  3 [(8, 8), (7, 7)]


In [8]:
partitionedRDD.take(2)

[[(2, 2), (4, 4), (6, 6), (8, 8)], [(1, 1), (3, 3), (5, 5), (7, 7)]]

In [9]:
pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (6, 6), (8, 8)], [(1, 1), (3, 3), (5, 5), (7, 7)]]

In [10]:
pairs.getNumPartitions()

2

In [11]:
import numpy as np
rddNP = sc.parallelize(np.array([1, 2, 3, 4, 5, 6, 7, 8]), 4)

In [12]:
def f(iterator): yield sum(iterator)
print(rddNP.mapPartitions(f).collect())

[3, 7, 11, 15]


In [13]:
pairs = sc.parallelize([1, 1, 1, 1, 1, 1, 1, 1])
pairs.collect()

[1, 1, 1, 1, 1, 1, 1, 1]

In [14]:
pairs=pairs.repartition(8)
pairs.collect()

[1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
pairs.getNumPartitions()

8

In [16]:
a = pairs.repartition(1).treeAggregate((1, 1), lambda x, y: (x[0]+y, x[1]+y), lambda x, y: (x[0]+y[0], x[1]+y[1]))
print(a)

(9, 9)


In [17]:
a =  pairs.repartition(4)\
.treeAggregate((1, 1), lambda x, y: (x[0]+y, x[1]+y) , lambda x, y: (x[0]+y[0],x[1]+y[1]))

print(a)

(12, 12)


In [18]:
a =  pairs.repartition(8)\
.treeAggregate((1, 1), lambda x, y: (x[0]+y, x[1]+y) , lambda x, y: (x[0]+y[0],x[1]+y[1]))

print(a)

(16, 16)


In [19]:
rddChars = sc.parallelize(['a', 'b', 'c', 'd', 'e', 'f', 'c', 'd'])
rddChars = rddChars.coalesce(4)

In [20]:
rddChars.glom().collect()

[['a', 'b', 'c', 'd'], ['e', 'f', 'c', 'd']]